In [1]:
# TensorFlow 变量是表示程序处理的共享持久状态的最佳方法。
# 与 tf.Tensor 对象不同，tf.Variable 存在于单个 session.run 调用的上下文之外。
# 在 TensorFlow 内部，tf.Variable 会存储持久性张量

In [2]:
import tensorflow as tf

In [3]:
# 指定名称和大小，并允许重复使用同名变量。可以定义重复利用层的模型。
my_variable = tf.get_variable("my_variable",[1,2,3])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# 变量默认类型为float32
my_int_variable = tf.get_variable("my_int_variable",[1,2,3],dtype=tf.int32,initializer=tf.zeros_initializer)

In [6]:
# 将变量初始化为tensor
# 当初始化器是 tf.Tensor 时，不应指定变量的形状，因为将使用初始化器张量的形状。
other_variable = tf.get_variable("other_variable", dtype=tf.int32, initializer=tf.constant([23,42]))

In [7]:
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)

In [9]:
# 使用变量
# v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1

In [18]:
# 要为变量赋值，请使用 assign、assign_add 方法以及 tf.Variable 类中的友元
# v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())

assignment = v.assign_add(1)
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    print(sess.run(assignment)) 

1.0


In [25]:
# 在事件发生后强制读取数据
#v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
    w = v.read_value()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w))

1.0


In [26]:
# 共享变量
# 1.显式传递tf.variable对象
# 2.将tf.variable对象隐式封装在tf.variable_scope对象内。
# 变量作用域允许您在调用隐式创建和使用变量的函数时控制变量重用。作用域还允许您以分层和可理解的方式命名变量。

In [27]:
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights",kernel_shape,initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases",bias_shape,initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv+biases)

In [28]:
# 无法重复调用
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5,5,32,32],bias_shape=[32])
#x = conv_relu(x, kernel_shape=[5,5,32,32],bias_shape=[32]) #fail

In [29]:
# 使用作用域
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        relu1 = conv_relu(input_images, [5,5,32,32],[32])
    with tf.variable_scope("conv2"):
        return conv_relu(relu1,[5,5,32,32],[32])

In [30]:
# 共享变量
# 添加reuse属性
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model",reuse=True):
    output2 = my_image_filter(input2)

In [35]:
# 使用reuse_variables函数
with tf.variable_scope("model1") as scope:
    output3 = my_image_filter(input1)
    scope.reuse_variables()
    output4 = my_image_filter(input2)

In [36]:
#由于依赖于作用域的确切字符串名称可能比较危险，因此也可以根据另一作用域初始化某个变量作用域：
with tf.variable_scope("model4") as scope:
    output1 = my_image_filter(input1)
with tf.variable_scope(scope, reuse=True):
    output2 = my_image_filter(input2)